# Mistral Models on Amazon Bedrock - Converse API Tests

This notebook uses the **Bedrock Converse API** (unified API) instead of invoke_model.

Models tested:
1. **Mistral Large 3** - Flagship multimodal model with reasoning
2. **Ministral 3B** - Efficient small model
3. **Ministral 8B** - Mid-size efficient model
4. **Ministral 14B** - Larger efficient model
5. **Voxtral Mini 3B** - Audio transcription model
6. **Voxtral Small 24B** - Advanced audio model
7. **Magistral Small 1.2** - Advanced reasoning model with vision (24B)

In [ ]:
%pip install --upgrade --quiet boto3 soundfile

In [ ]:
import boto3
import json
import base64

# Initialize Bedrock Runtime client
bedrock_runtime = boto3.client('bedrock-runtime', region_name='us-west-2')

## Helper Functions

In [ ]:
def converse_with_mistral(model_id, messages, system=None, max_tokens=2048, temperature=0.7, top_p=None, tools=None):
    """Invoke Mistral models using Converse API"""
    request = {
        "modelId": model_id,
        "messages": messages,
        "inferenceConfig": {
            "maxTokens": max_tokens,
            "temperature": temperature
        }
    }
    
    if top_p is not None:
        request["inferenceConfig"]["topP"] = top_p
    
    if system:
        request["system"] = [{"text": system}]
    
    if tools:
        request["toolConfig"] = {
            "tools": tools,
            "toolChoice": {"auto": {}}
        }
    
    response = bedrock_runtime.converse(**request)
    return response

def print_response(response):
    """Pretty print Converse API response"""
    if 'output' in response:
        message = response['output']['message']
        for content in message['content']:
            if 'text' in content:
                print(content['text'])
            elif 'toolUse' in content:
                tool_use = content['toolUse']
                print(f"Tool: {tool_use['name']}")
                print(f"Input: {json.dumps(tool_use['input'], indent=2)}")
    
    # Print usage stats
    if 'usage' in response:
        usage = response['usage']
        print(f"\n[Tokens - Input: {usage['inputTokens']}, Output: {usage['outputTokens']}, Total: {usage['totalTokens']}]")
    
    if 'stopReason' in response:
        print(f"[Stop Reason: {response['stopReason']}]")

## 1. Mistral Large 3 - Flagship Model Tests

### 1.1 Basic Text Generation

In [ ]:
model_id = "mistral.mistral-large-3-675b-instruct"

messages = [{
    "role": "user",
    "content": [{"text": "Explain quantum computing in 3 sentences."}]
}]

response = converse_with_mistral(model_id, messages, max_tokens=500)
print("=== Mistral Large 3 - Basic Generation ===")
print_response(response)

### 1.2 Reasoning Use Case with Extended Thinking

In [ ]:
messages = [{
    "role": "user",
    "content": [{"text": "A bat and ball cost $1.10 total. The bat costs $1 more than the ball. How much does the ball cost? Think step by step."}]
}]

# Lower temperature for reasoning tasks
response = converse_with_mistral(model_id, messages, max_tokens=1000, temperature=0.3)
print("\n=== Mistral Large 3 - Reasoning ===")
print_response(response)

### 1.3 Tool Use / Function Calling

In [ ]:
# Define tools using Converse API format
tools = [{
    "toolSpec": {
        "name": "get_weather",
        "description": "Get current weather for a location",
        "inputSchema": {
            "json": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "City name"
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"]
                    }
                },
                "required": ["location"]
            }
        }
    }
}]

messages = [{
    "role": "user",
    "content": [{"text": "What's the weather in Paris and Tokyo?"}]
}]

response = converse_with_mistral(model_id, messages, tools=tools)
print("\n=== Mistral Large 3 - Tool Use ===")
print_response(response)

### 1.4 Multi-turn Conversation

In [ ]:
messages = [
    {
        "role": "user",
        "content": [{"text": "What are the three laws of robotics?"}]
    },
    {
        "role": "assistant",
        "content": [{"text": "The Three Laws of Robotics by Isaac Asimov are: 1) A robot may not injure a human or allow harm through inaction, 2) A robot must obey human orders unless conflicting with First Law, 3) A robot must protect itself unless conflicting with First or Second Law."}]
    },
    {
        "role": "user",
        "content": [{"text": "Give me a scenario where these laws conflict."}]
    }
]

response = converse_with_mistral(model_id, messages)
print("\n=== Mistral Large 3 - Multi-turn ===")
print_response(response)

### 1.5 Vision - Image Understanding

Mistral Large 3 supports multimodal input with Converse API.

In [ ]:
# Load image from local repo
with open('Battle.png', 'rb') as f:
    image_data = f.read()

# Converse API format: use raw bytes, not base64
messages = [{
    "role": "user",
    "content": [
        {
            "image": {
                "format": "png",
                "source": {"bytes": image_data}
            }
        },
        {"text": "What is in this image? Describe it in detail."}
    ]
}]

response = converse_with_mistral(model_id, messages, max_tokens=1000)
print("\n=== Mistral Large 3 - Vision (Converse API) ===")
print_response(response)

### 1.6 With System Prompt

In [ ]:
system_prompt = "You are a helpful AI assistant that provides concise, accurate answers. Always cite sources when possible."

messages = [{
    "role": "user",
    "content": [{"text": "What is the speed of light?"}]
}]

response = converse_with_mistral(model_id, messages, system=system_prompt)
print("\n=== Mistral Large 3 - With System Prompt ===")
print_response(response)

## 2. Ministral 3B - Efficient Small Model

In [ ]:
model_id = "mistral.ministral-3-3b-instruct"

# Code generation
messages = [{"role": "user", "content": [{"text": "Write a Python function to calculate fibonacci numbers."}]}]
response = converse_with_mistral(model_id, messages, max_tokens=500)
print("=== Ministral 3B - Code Generation ===")
print_response(response)


In [ ]:

# Tool use test
tools = [{
    "toolSpec": {
        "name": "calculate",
        "description": "Perform mathematical calculation",
        "inputSchema": {
            "json": {
                "type": "object",
                "properties": {
                    "expression": {"type": "string", "description": "Math expression"}
                },
                "required": ["expression"]
            }
        }
    }
}]

messages = [{"role": "user", "content": [{"text": "What is 234 * 567?"}]}]
response = converse_with_mistral(model_id, messages, tools=tools)
print("\n=== Ministral 3B - Tool Use ===")
print_response(response)

In [ ]:
response

In [ ]:
# Load image from local repo
with open('Battle.png', 'rb') as f:
    image_data = f.read()

# Converse API format: use raw bytes, not base64
messages = [{
    "role": "user",
    "content": [
        {
            "image": {
                "format": "png",
                "source": {"bytes": image_data}
            }
        },
        {"text": "What is in this image? Describe it in detail."}
    ]
}]

response = converse_with_mistral(model_id, messages, max_tokens=1000)
print("\n=== Ministral 3B - Vision (Converse API) ===")
print_response(response)

## 3. Ministral 8B - Mid-Size Model

In [ ]:
model_id = "mistral.ministral-3-8b-instruct"

# Text analysis task
messages = [{
    "role": "user",
    "content": [{"text": "Analyze the sentiment and extract key entities from: 'Apple Inc. released iPhone 15 in September 2023, receiving positive reviews from tech enthusiasts.'"}]
}]

response = converse_with_mistral(model_id, messages)
print("=== Ministral 8B - NLP Task ===")
print_response(response)


In [ ]:
# Multi-tool use
tools = [
    {
        "toolSpec": {
            "name": "search_database",
            "description": "Search customer database",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "query": {"type": "string"}
                    },
                    "required": ["query"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "send_email",
            "description": "Send email to customer",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "to": {"type": "string"},
                        "subject": {"type": "string"},
                        "body": {"type": "string"}
                    },
                    "required": ["to", "subject", "body"]
                }
            }
        }
    }
]

messages = [{"role": "user", "content": [{"text": "Find customer John Doe and send him a reminder email about his pending order."}]}]
response = converse_with_mistral(model_id, messages, tools=tools)
print("\n=== Ministral 8B - Multi-Tool Use ===")
print_response(response)

In [ ]:
# Load image from local repo
with open('Battle.png', 'rb') as f:
    image_data = f.read()

# Converse API format: use raw bytes, not base64
messages = [{
    "role": "user",
    "content": [
        {
            "image": {
                "format": "png",
                "source": {"bytes": image_data}
            }
        },
        {"text": "What is in this image? Describe it in detail."}
    ]
}]

response = converse_with_mistral(model_id, messages, max_tokens=1000)
print("\n=== Ministral 8B - Vision (Converse API) ===")
print_response(response)

## 4. Ministral 14B - Larger Efficient Model

In [ ]:
model_id = "mistral.ministral-3-14b-instruct"

# Complex reasoning task
messages = [{
    "role": "user",
    "content": [{"text": "Design a database schema for an e-commerce platform with users, products, orders, and reviews. Explain your design choices."}]
}]

response = converse_with_mistral(model_id, messages, max_tokens=1500)
print("=== Ministral 14B - Complex Task ===")
print_response(response)


In [ ]:
# Load image from local repo
with open('Battle.png', 'rb') as f:
    image_data = f.read()

system_prompt = "Analyze images carefully and reason step by step using [THINK] tags."

messages = [{
    "role": "user",
    "content": [
        {
            "image": {
                "format": "png",
                "source": {"bytes": image_data}
            }
        },
        {"text": "Analyze this image. What do you observe?"}
    ]
}]

response = converse_with_mistral(model_id, messages, system=system_prompt, max_tokens=2048)
print("\n=== Ministral 14B - (Converse API) ===")
print_response(response)

## 5. Voxtral Models - Audio Transcription

✅ **Voxtral audio models WORK with proper audio format!**

**Required audio format:**
- Mono channel (1 channel)
- 16kHz sample rate
- MP3 format with low bitrate (32k)

**Note:** For Voxtral with Converse API, we still use the invoke_model approach since Converse API has limited audio support. The audio conversion helper works for both APIs.

In [ ]:
def convert_audio_for_voxtral(audio_file_or_bytes):
    """Convert audio to Voxtral-compatible format: mono, 16kHz, MP3"""
    from pydub import AudioSegment
    import io
    
    # Load audio (handles MP3, WAV, OGG, etc.)
    if isinstance(audio_file_or_bytes, bytes):
        audio = AudioSegment.from_file(io.BytesIO(audio_file_or_bytes))
    else:
        # Detect format from file extension
        if audio_file_or_bytes.endswith('.mp3'):
            audio = AudioSegment.from_mp3(audio_file_or_bytes)
        elif audio_file_or_bytes.endswith('.ogg'):
            audio = AudioSegment.from_ogg(audio_file_or_bytes)
        elif audio_file_or_bytes.endswith('.wav'):
            audio = AudioSegment.from_wav(audio_file_or_bytes)
        else:
            audio = AudioSegment.from_file(audio_file_or_bytes)
    
    # Convert to mono (1 channel)
    audio_mono = audio.set_channels(1)
    
    # Convert to 16kHz sample rate
    audio_mono = audio_mono.set_frame_rate(16000)
    
    # Export to MP3 bytes with low bitrate
    mp3_io = io.BytesIO()
    audio_mono.export(mp3_io, format='mp3', bitrate='32k')
    
    return mp3_io.getvalue()


def invoke_voxtral(model_id, audio_data, prompt="", max_tokens=2048):
    """Invoke Voxtral models with audio - uses invoke_model since Converse API has limited audio support"""
    # Encode audio to base64
    audio_base64 = base64.b64encode(audio_data).decode('utf-8')
    
    # Build content with audio
    content = [{
        "type": "input_audio",
        "input_audio": {
            "data": audio_base64,
            "format": "mp3"
        }
    }]
    
    # Add text prompt if provided
    if prompt:
        content.append({
            "type": "text",
            "text": prompt
        })
    
    body = {
        "messages": [{
            "role": "user",
            "content": content
        }],
        "max_tokens": max_tokens
    }
    
    response = bedrock_runtime.invoke_model(
        modelId=model_id,
        body=json.dumps(body)
    )
    return json.loads(response['body'].read())


# Example usage
print("=== Voxtral Mini - Audio Transcription ===")
print("# Convert and transcribe audio")
print("# audio_data = convert_audio_for_voxtral('obama.mp3')")
print("# response = invoke_voxtral('mistral.voxtral-mini-3b-2507', audio_data, prompt='Summarize this audio')")
print("# print(response['choices'][0]['message']['content'])")

In [ ]:
def invoke_voxtral_converse(model_id, audio_data, prompt="", max_tokens=2048):
  """
  Attempt to use Converse API with audio
  WARNING: May not work - Converse API has limited audio support for Voxtral
  """
  # Build content with audio as document
  content = [{
          "audio": {
              "format": "mp3",
              "source": {
                  "bytes": audio_data  # Raw bytes
              }
          }
      }]

  # Add text prompt if provided
  if prompt:
      content.append({
          "text": prompt
      })

  response = bedrock_runtime.converse(
      modelId=model_id,
      messages=[{
          "role": "user",
          "content": content
      }],
      inferenceConfig={
          "maxTokens": max_tokens
      }
  )

  return response

# Usage
audio_data = convert_audio_for_voxtral('obama.mp3')
response = invoke_voxtral_converse('mistral.voxtral-small-24b-2507', audio_data, prompt='Transcribe')

In [ ]:
response['output']['message']

## 6. Magistral Small 1.2 - Advanced Reasoning Model

### 6.1 Advanced Reasoning with THINK Tokens

In [ ]:
model_id = "mistral.magistral-small-2509"

# system_prompt = "You are a helpful assistant that thinks step by step. Use [THINK] and [/THINK] tokens to show your reasoning process before providing the final answer."

system_prompt = """First draft your thinking process (inner monologue) until you arrive at a response. 
Format your response using Markdown, and use LaTeX for any mathematical equations. 
Write both your thoughts and the response in the same language as the input.

Your thinking process must follow the template below:[THINK]Your thoughts or/and draft, 
like working through an exercise on scratch paper. Be as casual and as long as you want until you are confident to generate the response. 
Use the same language as the input.[/THINK]Here, provide a self-contained response.
"""

messages = [{
    "role": "user",
    "content": [{"text": "A farmer has 17 sheep, and all but 9 die. How many sheep are left?"}]
}]

response = converse_with_mistral(model_id, messages, system=system_prompt, max_tokens=2048, temperature=0.7)
print("=== Magistral Small - Basic Reasoning ===")
print_response(response)

In [ ]:
response

In [ ]:
# Image analysis with Converse API
with open('Battle.png', 'rb') as f:
    image_data = f.read()

messages = [{
    "role": "user",
    "content": [
        {
            "image": {
                "format": "png",
                "source": {"bytes": image_data}
            }
        },
        {"text": "What action do you think I should take in this situation? List all the possible actions and explain why you think they are good or bad."}
    ]
}]

response = converse_with_mistral(
    model_id='mistral.magistral-small-2509',
    messages=messages,
    max_tokens=2048,
    temperature=0.7,
    top_p=0.95
)
print("\n=== Magistral Small - Document Analysis (Converse API) ===")
print_response(response)

## Converse API Benefits

Key advantages of using the Converse API:

1. **Unified Interface**: Consistent API across all models
2. **Built-in Token Tracking**: Automatic usage statistics (inputTokens, outputTokens, totalTokens)
3. **System Prompts**: Dedicated parameter for system instructions
4. **Inference Config**: Centralized parameter management (maxTokens, temperature, topP)
5. **Tool Configuration**: Standardized tool/function calling with toolChoice options
6. **Stop Reason**: Explicit indication of why generation stopped
7. **Multi-modal Support**: Native support for text and images in content array
8. **Simplified Error Handling**: Consistent response structure

## Summary

**Text Models Tested:**
- **Mistral Large 3**: Complex reasoning, multi-turn conversations, tool use, system prompts
- **Ministral 3B/8B/14B**: Efficient models with tool use and varying capabilities
- **Magistral Small 1.2**: Advanced reasoning with [THINK] tokens, vision, optimized configs

**Audio Models:**
- **Voxtral Mini/Small**

**Key Capabilities Tested:**
- Text generation with inferenceConfig
- System prompts for context and instructions
- Tool use with toolSpec format
- Multi-turn conversations
- Vision reasoning (images + text)
- Reasoning with [THINK] tokens
- Performance benchmarking with token tracking